In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [2]:
# Define transforms for the training and testing data
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert images to grayscale
        transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
        transforms.ToTensor(),  # Convert images to PyTorch tensors
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
    ]),
}

In [3]:
# Load the datasets
train_dataset = datasets.ImageFolder(root='Data_Processed/train', transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root='Data_Processed/test', transform=data_transforms['test'])

In [4]:
# Create the dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check the classes
class_names = train_dataset.classes
print(f'Classes: {class_names}')

Classes: ['COVID19', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  # 1 input channel for grayscale images
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cpu


In [7]:
# Instantiate the model
num_classes = len(class_names)
model = SimpleCNN(num_classes).to(device)

In [8]:
# Load the saved state dictionary into the model (to continue training from previously trained model)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [10]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

epochs = 5 # Number of training epochs

# Training loop
for epoch in range(epochs):  # Adjust the number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

Epoch 1/5: 100%|██████████████████████████████████████████████████████████████████| 198/198 [03:54<00:00,  1.18s/batch]


Epoch 1, Loss: 0.3624674655083153


Epoch 2/5: 100%|██████████████████████████████████████████████████████████████████| 198/198 [03:50<00:00,  1.16s/batch]


Epoch 2, Loss: 0.2442496054137897


Epoch 3/5: 100%|██████████████████████████████████████████████████████████████████| 198/198 [03:51<00:00,  1.17s/batch]


Epoch 3, Loss: 0.16622955510110565


Epoch 4/5: 100%|██████████████████████████████████████████████████████████████████| 198/198 [03:43<00:00,  1.13s/batch]


Epoch 4, Loss: 0.14049803553798207


Epoch 5/5: 100%|██████████████████████████████████████████████████████████████████| 198/198 [03:49<00:00,  1.16s/batch]

Epoch 5, Loss: 0.10437775512621032
Finished Training


In [11]:
# Save the trained model
torch.save(model.state_dict(), 'model.pth')

In [12]:
# Initialize counters for each class
correct_per_class = np.zeros(num_classes)
total_per_class = np.zeros(num_classes)

# Initialize counters for overall accuracy
correct_total = 0
total_samples = 0

# Evaluation loop
model.eval()

with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc='Evaluating', unit='batch'):
        # Move inputs and labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update the total count per class and overall
        for label in labels:
            total_per_class[label.item()] += 1
            total_samples += 1
        
        # Update the correct count per class and overall
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_per_class[label.item()] += 1
                correct_total += 1

# Calculate accuracy for each class
for i in range(num_classes):
    if total_per_class[i] > 0:
        accuracy = 100 * correct_per_class[i] / total_per_class[i]
        print(f'Accuracy of class {class_names[i]}: {accuracy:.2f}%')

# Calculate overall accuracy
overall_accuracy = 100 * correct_total / total_samples
print(f'Total model accuracy: {overall_accuracy:.2f}%')

Evaluating: 100%|███████████████████████████████████████████████████████████████████| 25/25 [00:27<00:00,  1.10s/batch]

Accuracy of class COVID19: 77.36%
Accuracy of class NORMAL: 38.03%
Accuracy of class PNEUMONIA: 98.97%
Accuracy of class TUBERCULOSIS: 97.56%
Total model accuracy: 77.43%
